##**Part 1 :: Creating training labels from XML file to train custom Spacy Model**



In [ ]:
!mkdir xml_files #Create a directory for interacting with operating system

In [ ]:
#Load the .zip into your directory and rename he path in the cell below

In [ ]:
!unzip Protokoll-Zionistenkongress-Basel_1897-0200.zip -d xml_files/ #unziü the zip file which contains all .xml files with the annotations of the Persons and Locations
                                

In [ ]:
#Import these packages and dependencies - have also a look at "requirements.txt"

In [ ]:
import os
import xml.etree.ElementTree as ET

In [ ]:
xml_files = os.listdir("xml_files")

In [ ]:
xml_files = sorted(xml_files)

In [ ]:
len(xml_files)

202

**Below cell is the logic for the parsing XML file and extracting label and converting it to spacy format.**

In [ ]:
final_all_ents_tuple = []
all_sentences_present = []

#looping over all the files
for j in range(len(xml_files)):

  print("processing file=================================== ", xml_files[j])
  
  mytree = ET.parse('xml_files/'+xml_files[j])
  myroot = mytree.getroot()

  for x in myroot[1][1]:
    if x.tag.endswith('TextLine'):
      if "person" in x.attrib['custom'] or "place" in x.attrib['custom']:
 
        ents = x.attrib['custom'].split(" ")[2:]
        print(ents)
        sentence = x[-1][0].text
        all_sentences_present.append(sentence)
 
        all_ents = []
 
        for i in range(0, len(ents)):
          if ents[i] in ['person', 'place']:
            if ents[i] == 'person':
               ent = 'PERSON'
            else:
              ent = 'LOC'
 
            a = int(ents[i+1].split(":")[1][:-1])

            ## following if-else condition is written as there are some labels which has 'continued:true' means there are more word belong to current word
            if ents[i+2].endswith("}"):
              b = int(ents[i+2].split(":")[1][:-2])
            else:
              try:
                i += 4
                b1 = int(a[i+1].split(":")[1][:-1])
                if ents[i+2].endswith("}"):
                  b2 = int(ents[i+2].split(":")[1][:-2])
                else:
                  b2 = int(ents[i+2].split(":")[1][:-1])
                b = b1 + b2
              except:
                i -= 4
                b = int(ents[i+2].split(":")[1][:-1])
 
            ent_tuple = (a, a+b, ent) #single tuple as per the format defined by spacy
            all_ents.append(ent_tuple)

        # following loop is written because in there are some samples which has overlapping range, this loop handles those overlapping words as they are already covered.
        all_ents_copy = all_ents.copy()
        for k in range(len(all_ents)-1):
          if all_ents[k][0] <= all_ents[k+1][0] <= all_ents[k][1] or all_ents[k][0] <= all_ents[k+1][1] <= all_ents[k][1]:
            try:
              del all_ents_copy[k+1]
            except:
              del all_ents_copy[k]

        final_tuple = (sentence, {'entities' : all_ents_copy})
        print(final_tuple)
        final_all_ents_tuple.append(final_tuple)  #this variable holds all the tuples from all the files
        print("=="*50)

Streaming output truncated to the last 5000 lines.
['person', '{offset:0;', 'length:6;}', 'person', '{offset:8;', 'length:12;}', 'person', '{offset:22;', 'length:18;}', 'person', '{offset:42;', 'length:11;}', 'person', '{offset:55;', 'length:8;}']
('Philos, Ibn Gabirols, Jehuda ben Halewys, Ben Maimons, Spinozas,', {'entities': [(0, 6, 'PERSON'), (8, 20, 'PERSON'), (22, 40, 'PERSON'), (42, 53, 'PERSON'), (55, 63, 'PERSON')]})
['person', '{offset:0;', 'length:6;}']
('Heines kennt diese Geldprotzen nicht, die alles geringschätzen, was', {'entities': [(0, 6, 'PERSON')]})
processing file===================================  Protokoll-Zionistenkongress-Basel_1897-0022.xml
['person', '{offset:39;', 'length:14;}']
('Präsident: Das Wort hat Herr Architekt Oscar Marmorek:', {'entities': [(39, 53, 'PERSON')]})
['person', '{offset:0;', 'length:14;}']
('Oscar Marmorek: Geliebte Brüder! Ein Wundertag ist der heutigé,', {'entities': [(0, 14, 'PERSON')]})
processing file===============================

In [ ]:
# storing all the labels in txt file 
with open("outfile.txt", "w") as outfile:
    outfile.write("\n".join(str(item) for item in final_all_ents_tuple))

##**Part 2 :: Custom training with Spacy**

In [ ]:
!pip show spacy

Name: spacy
Version: 2.2.4
Summary: Industrial-strength Natural Language Processing (NLP) in Python
Home-page: https://spacy.io
Author: Explosion
Author-email: contact@explosion.ai
License: MIT
Location: /usr/local/lib/python3.7/dist-packages
Requires: catalogue, plac, tqdm, thinc, setuptools, requests, murmurhash, blis, wasabi, srsly, numpy, cymem, preshed
Required-by: fastai, en-core-web-sm


In [ ]:
from __future__ import unicode_literals, print_function
import plac
import random
from pathlib import Path
import spacy
from tqdm import tqdm

In [ ]:
!mkdir de_spacy_custom

In [ ]:
model = None
output_dir=Path("/content/de_spacy_custom") #output folder in which trained model will be stored
n_iter=100 #number of training epochs

In [ ]:
if model is not None:
    nlp = spacy.load(model)  
    print("Loaded model '%s'" % model)
else:
    nlp = spacy.blank('de')  
    print("Created blank 'de' model")

#set up the pipeline

if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe(ner, last=True)
else:
    ner = nlp.get_pipe('ner')

Created blank 'de' model


**Below cell is for spacy training code**

**It took more than 2 hours on google colab to complete training for the 100 epochs.**

In [ ]:
for _, annotations in final_all_ents_tuple:
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])

other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):  # only train NER
    optimizer = nlp.begin_training()
    for itn in range(n_iter):
        random.shuffle(final_all_ents_tuple)
        losses = {}
        for text, annotations in tqdm(final_all_ents_tuple):
            nlp.update(
                [text],  
                [annotations],  
                drop=0.5,  
                sgd=optimizer,
                losses=losses)
        print(losses)

100%|██████████| 1702/1702 [00:50<00:00, 33.47it/s]


{'ner': 3468.15108129049}


100%|██████████| 1702/1702 [01:01<00:00, 27.49it/s]


{'ner': 2360.619252815709}


100%|██████████| 1702/1702 [01:07<00:00, 25.18it/s]


...

{'ner': 360.7215248082787}


100%|██████████| 1702/1702 [01:24<00:00, 20.14it/s]


{'ner': 317.2809933649003}


100%|██████████| 1702/1702 [01:25<00:00, 20.02it/s]

{'ner': 330.6965194955011}


In [ ]:
#saving trained model in directory
if output_dir is not None:
    output_dir = Path(output_dir)
    if not output_dir.exists():
        output_dir.mkdir()
    nlp.to_disk(output_dir)
    print("Saved model to", output_dir)

Saved model to /content/de_spacy_custom


In [ ]:
#sample inference using trained model
for text, _ in final_all_ents_tuple[:5]:
    doc = nlp(text)
    print('Entities', [(ent.text, ent.label_) for ent in doc.ents])

Entities [('London', 'LOC')]
Entities [('Waissmann', 'PERSON'), ('Moritz Guttmann', 'PERSON'), ('Josef Herzler, Elias Schwarz', 'PERSON')]
Entities [('Lamprecht', 'PERSON')]
Entities [('Türkei', 'LOC')]
Entities [('Daniel Spielmann', 'PERSON'), ('Julius Rischou', 'PERSON'), ('Alois Eisenstein', 'PERSON'), ('Max Fuchs', 'PERSON')]


##**Part 3 :: Inference of Custom trained model on test data - change this txt file and/or path to set the test dataset**

In [ ]:
import pandas as pd
test_df = pd.read_csv('03_Protokoll-Zionistenkongress-Basel_1899.txt', delimiter = "\n", header=None, names=["text"])

In [ ]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14740 entries, 0 to 14739
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    14740 non-null  object
dtypes: object(1)
memory usage: 115.3+ KB


In [ ]:
test_df.head(30)

In [ ]:
all_persons = []
all_locations = []

for jj in range(len(test_df)):
  doc = nlp(test_df['text'][jj])
  persons = [ent.text for ent in doc.ents if ent.label_ == 'PERSON']
  locations = [ent.text for ent in doc.ents if ent.label_ == 'LOC']
  all_persons.append(persons)
  all_locations.append(locations)

In [ ]:
test_df['Custom-trained_Spacy_Person'] = pd.Series(all_persons)
test_df['Custom-trained_Spacy_Location'] = pd.Series(all_locations)

In [ ]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14740 entries, 0 to 14739
Data columns (total 3 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   text                           14740 non-null  object
 1   Custom-trained_Spacy_Person    14740 non-null  object
 2   Custom-trained_Spacy_Location  14740 non-null  object
dtypes: object(3)
memory usage: 345.6+ KB


In [ ]:
test_df.head(30)

In [ ]:
test_df.tail(30)

In [ ]:
##**These are the results of the own trained model - saves as "Custom_NER_inference_results.csv" - change directory and/or name if needed**

In [ ]:
test_df.to_csv('Custom_NER_inference_results.csv',index=False) #saving inference results of custom trained model


##**Part 4 :: Using pre-trained German spacy Large model to detect entity**

In [ ]:
!pip install -U spacy

     |████████████████████████████████| 6.2 MB 23.1 MB/s 
     |████████████████████████████████| 653 kB 50.4 MB/s 
     |████████████████████████████████| 181 kB 42.4 MB/s 
     |████████████████████████████████| 42 kB 1.5 MB/s 
     |████████████████████████████████| 457 kB 58.4 MB/s 
     |████████████████████████████████| 10.1 MB 46.5 MB/s 
     |████████████████████████████████| 58 kB 6.8 MB/s 
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.2.0
    Uninstalling typing-extensions-4.2.0:
      Successfully uninstalled typing-extensions-4.2.0
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: smart-open
    Found existing installation: smart-open 6.0.0
    

In [ ]:
!python -m spacy download de_core_news_lg

     |████████████████████████████████| 567.8 MB 25 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_lg')


In [ ]:
import spacy

In [ ]:
import pandas as pd

In [ ]:
nlp = spacy.load("de_core_news_lg")

In [ ]:
df = pd.read_csv("Custom_NER_inference_results.csv")

In [ ]:
all_persons = []
all_locations = []

for jj in range(len(df)):
  doc = nlp(df['text'][jj])
  persons = [ent.text for ent in doc.ents if ent.label_ == 'PER']
  locations = [ent.text for ent in doc.ents if ent.label_ == 'LOC']
  all_persons.append(persons)
  all_locations.append(locations)

In [ ]:
df['Pre-trained_Spacy_Person'] = pd.Series(all_persons)
df['Pre-trained_Spacy_Location'] = pd.Series(all_locations)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14740 entries, 0 to 14739
Data columns (total 5 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   text                           14740 non-null  object
 1   Custom-trained_Spacy_Person    14740 non-null  object
 2   Custom-trained_Spacy_Location  14740 non-null  object
 3   Pre-trained_Spacy_Person       14740 non-null  object
 4   Pre-trained_Spacy_Location     14740 non-null  object
dtypes: object(5)
memory usage: 575.9+ KB


In [ ]:
df.tail(50)

,text,Custom-trained_Spacy_Person,Custom-trained_Spacy_Location,Pre-trained_Spacy_Person,Pre-trained_Spacy_Location
14690,asui. Verein „Jerusalem.,[],['Jerusalem'],[],[]
14691,Russland.,[],['Russland'],[],[]
14692,Aus 174 Orten liefen 256 Zuschriften und Teleg...,[],['Telegramme'],[],[]
14693,personen und 213 Gruppen und Corporationen ein.,[],['Gruppen'],[],[]
14694,Schweiz.,[],['Schweiz'],[],[]
14695,Bern. Zionistische Studierende.,[],['Bern'],[],[]
14696,Zionistische Ortsgruppe.,[],[],[],[]
14697,Genf. Bernard Mochenson.,['Genf. Bernard Mochenson'],[],[Bernard Mochenson.],[Genf]
14698,Zionistische Ortsgruppe.,[],[],[],[]
14699,Locle. Dr. Beredka.,['Dr. Beredka'],['Locle'],[Locle.],[]


In [ ]:
df.to_csv("Custom_NER_All_Inference_results.csv", index=False) #saving final results which has results of custom model and pre-trained spacy large model.